<a href="https://colab.research.google.com/github/Locrian24/csc421-project-stress-classification/blob/main/csc421_project_cnn_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!cp /content/drive/MyDrive/WESAD_RespiBAN_train.tar .
!cp /content/drive/MyDrive/WESAD_RespiBAN_test.tar .

In [10]:
!pip install webdataset

In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import webdataset as wds

In [12]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 40
NUM_EPOCHS = 100

In [13]:
!pip install pytorch_lightning

  Using cached pytorch_lightning-1.6.0-py3-none-any.whl (582 kB)
  Using cached torchmetrics-0.7.3-py3-none-any.whl (398 kB)
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Using cached pyDeprecate-0.3.2-py3-none-any.whl (10 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Using cached fsspec-2022.3.0-py3-none-any.whl (136 kB)
  Using cached aiohttp-3.8.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 271 kB 52.4 MB/s 
  Using cached frozenlist-1.3.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalli

In [14]:
import pytorch_lightning as pl

In [15]:
import logging

In [25]:
from torchmetrics.functional import accuracy

In [16]:
def conv_block():
  block = nn.Sequential(
      nn.Conv1d(in_channels=1, out_channels=8, kernel_size=32, stride=2, padding=1),
      nn.ReLU(),
      nn.MaxPool1d(4, 4),
      nn.Conv1d(in_channels=8, out_channels=16, kernel_size=7, stride=2, padding=1),
      nn.ReLU(),
      nn.MaxPool1d(4, 4),
      nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=2),
      nn.ReLU(),
      nn.MaxPool1d(2, 2),
  )

  return block

In [40]:
class BinaryStressClassifier(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.conv_block = conv_block()
    self.fc1 = nn.Sequential(
        nn.Linear(6912, 32),
        nn.ReLU()
    )
    self.fc2 = nn.Sequential(
        nn.Linear(32, 16),
        nn.ReLU()
    )
    self.fc3 = nn.Sequential(
        nn.Linear(16, 1),
        nn.Sigmoid()
    )

    self.criterion = nn.BCELoss()
  
  def forward(self, X):
    X = X.float()

    ECG_x = self.conv_block(X[:,:,0])
    EDA_x = self.conv_block(X[:,:,1])
    EMG_x = self.conv_block(X[:,:,2])
    Temp_x = self.conv_block(X[:,:,3])
    Resp_x = self.conv_block(X[:,:,4])
    ACC_x_x = self.conv_block(X[:,:,5])
    ACC_y_x = self.conv_block(X[:,:,6])
    ACC_z_x = self.conv_block(X[:,:,7])

    ECG_x = torch.flatten(ECG_x, 1)
    EDA_x = torch.flatten(EDA_x, 1)
    EMG_x = torch.flatten(EMG_x, 1)
    Temp_x = torch.flatten(Temp_x, 1)
    Resp_x = torch.flatten(Resp_x, 1)
    ACC_x_x = torch.flatten(ACC_x_x, 1)
    ACC_y_x = torch.flatten(ACC_y_x, 1)
    ACC_z_x = torch.flatten(ACC_z_x, 1)

    x = torch.cat((ECG_x,EDA_x,EMG_x,Temp_x,Resp_x,ACC_x_x,ACC_y_x,ACC_z_x), dim=1)

    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)

    return x

  def train_dataloader(self):
    dataset = (
      wds.WebDataset('WESAD_RespiBAN_train.tar')
        .shuffle(4 * BATCH_SIZE)
        .decode()
        .to_tuple("input.npy", "label.cls")
        .map_tuple(transforms.ToTensor(), lambda x: 1 if x == 2 else 0 ) # Map labels for binary classification (2 = "stress")
    )

    return torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE)

  def test_dataloader(self):
    dataset = (
      wds.WebDataset('WESAD_RespiBAN_test.tar')
        .decode()
        .to_tuple("input.npy", "label.cls")
        .map_tuple(transforms.ToTensor(), lambda x: 1 if x == 2 else 0 ) # Map labels for binary classification (2 = "stress")
    )

    return torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-4)

  def training_step(self, batch, batch_idx):
    inputs, labels = batch
    outputs = self.forward(inputs)
    labels = labels.unsqueeze(1).float()
    loss = self.criterion(outputs, labels)

    return dict(loss = loss)

  def test_step(self, batch, batch_idx):
    inputs, labels = batch
    outputs = self.forward(inputs)
    acc = accuracy(outputs, labels)

    self.log("accuracy:", acc, prog_bar=True)

classifier = BinaryStressClassifier()


In [27]:
trainer = pl.Trainer(max_epochs=NUM_EPOCHS, accelerator="auto")
logging.getLogger("lightning").setLevel(logging.WARNING)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(classifier)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | conv_block | Sequential | 2.7 K 
1 | fc1        | Sequential | 221 K 
2 | fc2        | Sequential | 528   
3 | fc3        | Sequential | 17    
4 | criterion  | BCELoss    | 0     
------------------------------------------
224 K     Trainable params
0         Non-trainable params
224 K     Total params
0.898     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [23]:
# trainer.save_checkpoint("cnn_1.ckpt")

In [41]:
model = BinaryStressClassifier.load_from_checkpoint("./cnn_1.ckpt")

In [42]:
test_trainer = pl.Trainer(accelerator="gpu", devices=1)
trainer.test(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        accuracy:           0.9498031735420227
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'accuracy:': 0.9498031735420227}]